In [1]:
import cv2
import numpy as np
import string
from tensorflow.keras.models import load_model

c:\Users\acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/

In [2]:
# ==========================
# 1. Load model
# ==========================
model = load_model("asl_sign_model.h5")

In [3]:
# Alphabet tanpa J dan Z (karena dataset tidak punya)
alphabet = list(string.ascii_uppercase)
alphabet.remove('J')
alphabet.remove('Z')

In [4]:
# ==========================
# 2. Ground Truth
# ==========================
# Huruf pada grid amer_sign2.png (4 baris x 7 kolom)
ground_truth = [
    ['A','B','C','D','E','F','G'],
    ['H','I','K','L','M','N','O'],
    ['P','Q','R','S','T','U','V'],
    ['W','X','Y','','','', '']  # baris terakhir hanya 3 huruf
]

In [5]:
# ==========================
# 3. Load gambar grid
# ==========================
img_path = "amer_sign2.png"
full_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

rows, cols = 4, 7
h, w = full_img.shape[0] // rows, full_img.shape[1] // cols

In [6]:
# ==========================
# 4. Loop per kotak huruf
# ==========================
predictions = []
total = 0
benar = 0

In [9]:
print("===== HASIL PREDIKSI (dengan preprocessing) =====")
for i in range(rows):
    row_preds = []
    for j in range(cols):
        if ground_truth[i][j] == '':
            row_preds.append('-')
            continue

        # Crop kotak huruf
        crop = full_img[i*h:(i+1)*h, j*w:(j+1)*w]

        # 🔹 Preprocessing tambahan
        # 1. Threshold (buat background hitam-putih jelas)
        _, crop = cv2.threshold(crop, 120, 255, cv2.THRESH_BINARY)

        # 2. Invert (agar tangan putih, background hitam)
        crop = cv2.bitwise_not(crop)

        # 3. Resize ke 28x28 dan normalisasi
        crop_resized = cv2.resize(crop, (28,28)) / 255.0
        crop_resized = crop_resized.reshape(1,28,28,1)

        # Prediksi
        pred = model.predict(crop_resized, verbose=0)
        label_pred = alphabet[np.argmax(pred)]
        row_preds.append(label_pred)

        # Hitung akurasi
        total += 1
        if label_pred == ground_truth[i][j]:
            benar += 1

    predictions.append(row_preds)
    print(" ".join(row_preds))

===== HASIL PREDIKSI (dengan preprocessing) =====
H M M Q R Q M
H B M Q R R V
H S Q Q R S M
M Q Q - - - -


In [10]:
# ==========================
# 5. Tampilkan Akurasi
# ==========================
akurasi = benar / total * 100
print(f"\nAkurasi: {benar}/{total} = {akurasi:.2f}%")


Akurasi: 2/48 = 4.17%
